# Rule Splitting & ElasticSearch Uploads, Queries
### LMI Capstone Team
### Summer Chambers | Steve Morris | Kaleb Shikur

## Setup

In [30]:
import pandas as pd
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
import regex as re
from bs4 import BeautifulSoup
import numpy as np
import requests #gets urls
import time
import json
import csv
import os
import sys
import string

In [31]:
# 173.79.72.92, port 9200
# host = '173.79.72.92'
endpoint = 'https://search-lmi-capstone-2-525zkk33t4z5iy6ozqd63ctgmq.us-east-1.es.amazonaws.com'

In [32]:
#es = Elasticsearch(endpoint, timeout = 45)
es = Elasticsearch(endpoint, timeout=600, max_retries=2, retry_on_timeout=True)



print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'eb7b3348d3c34ce7dc22f263930c51ad', 'cluster_name': '846033058400:lmi-capstone-2', 'cluster_uuid': 'QzTffcNgStmet053IRSP2w', 'version': {'number': '7.9.1', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-11-03T09:54:32.349659Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


## Define Rule Splits

In [33]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

In [34]:
startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}

In [35]:
def splitRule_headers(rule_url, startdict):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = []
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks.append({"section": key, "text": (value[0]+split_further[0])}) #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split

    return rulechunks

In [37]:
test_header_split = splitRule_headers(rule_url, startdict)
headers_id_list = [doc["section"] for doc in test_header_split]

In [8]:
def splitRule_line_hybrid(rule_url):
    new_rule_chunks = []
    chunks = splitRule_headers(rule_url, startdict)
    for doc in chunks:
        paragraphs = doc["text"].split('\r\n')
        #add new lines while under 6000 characters
        for i in range(len(paragraphs) - 1):
            while i < (len(paragraphs)-1) and len(paragraphs[i]) < 6000:
                paragraphs[i] += paragraphs[i+1]
                del(paragraphs[i+1])
        for i in range(len(paragraphs)):
            new_rule_chunks.append({"section": doc["section"]+str(i), "text": paragraphs[i]})
    return new_rule_chunks

In [13]:
result = splitRule_line_hybrid(rule_url)
lengths = {chunk["section"]: len(chunk["text"]) for chunk in result}

## Uploading Rule Splits to ES

In [9]:
def rulesplit_toES(rulechunks, es_index):
    for chunk in rulechunks:
            res = es.index(index=es_index, id=chunk["section"], body=chunk, doc_type='_doc')
            es.indices.refresh(index=es_index)
    print("Last id uploaded:", chunk["section"])

In [59]:
rulesplit_1shard = splitRule_headers(rule_url)
upload_1shard = rulesplit_toES(rulesplit_1shard, "headers_1shard2")

Last id uploaded: f3


In [151]:
rulesplit_1shard = splitRule_line_hybrid(rule_url)
upload_1shard = rulesplit_toES(rulesplit_1shard, "hybrid_1shard")

Last id uploaded: f31


In [31]:
rulesplit_test1 = splitRule_headers(rule_url, startdict)
test_upload1 = rulesplit_toES(rulesplit_test1, "headers_broadsyn_custstop")

Last id uploaded: f3


In [18]:
rulesplit_test2 = splitRule_headers(rule_url, startdict)
test_upload2 = rulesplit_toES(rulesplit_test2, "headers_narrowsyn_custstop")

Last id uploaded: f3


In [12]:
rulesplit_test2 = splitRule_line_hybrid(rule_url)
test_upload2 = rulesplit_toES(rulesplit_test2, "hybrid_custom")

Last id uploaded: f31


In [10]:
rulesplit_test2 = splitRule_headers(rule_url, startdict)
test_upload2 = rulesplit_toES(rulesplit_test2, "headers_expsyn_basestop")

Last id uploaded: f3


In [9]:
rulesplit_test2 = splitRule_headers(rule_url, startdict)
test_upload2 = rulesplit_toES(rulesplit_test2, "headers_nosyn_basestop")

Last id uploaded: f3


In [14]:
rulesplit_test2 = splitRule_headers(rule_url, startdict)
test_upload2 = rulesplit_toES(rulesplit_test2, "headers_expsyn_comsyn_basestop")

Last id uploaded: f3


### Index Utilities:

In [ ]:
##es.indices.refresh(index="index_name")

In [52]:
##es.indices.delete(index='sanity_check', ignore=[400, 404])

{'acknowledged': True}

## Retrieve Comment Documents

### Scrape comments

In [10]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"

In [11]:
def retrieve_comments(comment_url):
    response = requests.get(comment_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.findAll("a")
    links = [tag["href"] for tag in a_tags]
    txt_links = [link for link in links if '.txt' in link]
    comments = {}
    for suffix in txt_links:
        comments[suffix] = requests.get(comment_url+suffix).text.lower()
        #print(f"scraping comment {suffix}")
    print(f"scraped {len(txt_links)} comments")
    return comments

In [ ]:
comments2018 = retrieve_comments(comment_url)

In [ ]:
comment_lengths = {key: len(value) for key, value in comments2018.items()}
sorted_comment_lengths = dict(sorted(comment_lengths.items(), key=lambda x: x[1]))

In [ ]:
comments2018 = {(key[14:20]): value for key, value in comments2018.items()}
sorted_keys = sorted(list(comments2018.keys()))

# now for each key in the list
for i in range(len(comments2018)-1):
    # get key at index i and key at index i+1 and compare them
    if sorted_keys[i+1][0:4] == sorted_keys[i][0:4]:
        comments2018[sorted_keys[i+1]] = comments2018[sorted_keys[i]] + comments2018[sorted_keys[i+1]]
        del(comments2018[sorted_keys[i]])

In [ ]:
comments2018 = {key[0:4]:value for key, value in comments2018.items()}

In [ ]:
len(comments2018)

### Write to .json

In [ ]:
comment_json = json.dumps(comments2018)
f = open('comments2018.json','w')
f.write(comment_json)
f.close()

### Read .json (avoid scraping comments)

In [17]:
with open('comments2018.json') as f:
    comments2018 = json.load(f)

## Remove duplicate comments, comments less than 30 characters long

In [122]:
reduced_comments2018 = {key: val for key, val in comments2018.items() if len(val) >= 30}
short_ones = {key: val for key, val in comments2018.items() if len(val) < 30}
list_reduced_comments2018 = list(reduced_comments2018.values())
len(reduced_comments2018)
short_ones #these seem okay to remove

{'0083': 'good luck everyone!',
 '0142': 'see attached file(s)',
 '0237': 'see attached file(s)\n',
 '0270': 'see attached file(s)\n',
 '0293': 'see attached file(s)',
 '0321': 'see attached file(s)\n\n',
 '0324': 'see attachement\n',
 '0329': 'see attached file(s)\n',
 '0342': 'see attached file(s)\n',
 '0447': 'see attached file(s)'}

In [123]:
from DeDuper import *

In [124]:
duplicates = getDupes(list_reduced_comments2018)

In [125]:
print(len(duplicates))
#duplicates

50


### Examine Duplicates

In [126]:
dupdict = {}
for i, dup in enumerate(duplicates):
    dupdict[i] = [list_reduced_comments2018[idx][0:120] for idx in dup]

In [139]:
#dupdict

### Remove Them

In [129]:
tb_deleted = []
for num, duplist in enumerate(duplicates):
    for idx, comment in enumerate(list_reduced_comments2018):
        if idx in duplist[:-1]:
            tb_deleted.append(comment)

In [130]:
len(tb_deleted)

363

In [131]:
unique_comments2018 = {key:value for key, value in reduced_comments2018.items() if value not in tb_deleted}

In [132]:
len(unique_comments2018)

94

### Write to .json

In [133]:
dump = json.dumps(unique_comments2018)
f = open('unique_comments2018.json','w')
f.write(dump)
f.close()

### Read .json (avoid scraping comments)

In [18]:
with open('unique_comments2018.json') as f:
    unique_comments2018 = json.load(f)

## Split up longer comments

In [101]:
long_comments = {key:value for key, value in unique_comments2018.items() if len(value) > 5000}

In [102]:
expanded = unique_comments2018.copy()
for key, value in long_comments.items():
    paragraphs = value.split('\n')
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs) - 1) and len(paragraphs[i]) < 4800:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])
    for i in range(len(paragraphs)):
        expanded[key+'_'+str(i)] = paragraphs[i]
expanded = {key:value for key, value in expanded.items() if key not in list(long_comments.keys())}

In [63]:
#{key:len(value) for key, value in expanded.items()}

## Remove \r, \n, \s, and any weird or non-ascii characters

In [25]:
expanded = {key:value.replace("\t", " ").replace("\r", " ").replace("\n", " ").replace("\s", " ") for key, value in expanded.items()}

In [26]:
for key, value in expanded.items():
    expanded[key] = ''.join(c for c in value if c in string.printable)

### Write to .json

In [27]:
dump = json.dumps(expanded)
f = open('expanded.json','w')
f.write(dump)
f.close()

### Read .json (avoid scraping comments)

In [12]:
with open('expanded.json') as f:
    expanded = json.load(f)

## Testing other basic API Features

Check to see if document_id exists in index

In [ ]:
es.exists(index="headers", id=40)

View term vectors for a document

In [62]:
es.termvectors(index="headers", id=6)

{'_index': 'headers',
 '_type': '_doc',
 '_id': '6',
 '_version': 1,
 'found': True,
 'took': 0,
 'term_vectors': {}}

See an explanation for a query's score

In [ ]:
es.explain(index="hybrid_6000", id=6, body={"query": test_comment0002_query}) 

## Querying ES

In [125]:
#Single query
def ES_search(es_index, querydict, search_type=None):
    if search_type:
        search = es.search(index=es_index, doc_type="_doc", body={"query": querydict}, search_type=search_type, size=5)
    else:
        search = es.search(index=es_index, doc_type="_doc", body={"query": querydict})
    test_dict = {}
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            test_dict[h['_id']]=h['_score']
    return test_dict

### Test a few types of queries

With "Simple Query String"

In [63]:
test_comment0002_query = {"simple_query_string": {"query": expanded['0002']}}

In [64]:
ES_search("headers_standard", test_comment0002_query)

{'a4c': 151.79832,
 'a6c': 148.1676,
 'a2': 77.816055,
 'a5b': 69.46873,
 'a5c': 63.691624,
 'a3': 62.509632,
 'a5d': 59.190556,
 'c3b': 57.04912,
 'd3d': 56.666798,
 'e4': 55.296684}

With "More Like This"

In [37]:
mlt_test0002 = {"more_like_this": {"fields": ["text"], "like": expanded["0004"]}}#, "min_term_freq": 1, "max_query_terms": 30}}

In [39]:
ES_search("headers_standard", mlt_test0002)

{'a7c': 4.9438677,
 'a4c': 4.86566,
 'a6c': 4.861052,
 'a6b': 4.762745,
 'a5b': 4.6482067,
 'e3': 4.563411,
 'a6d3': 4.2192745,
 'a3': 4.1510067,
 'a4b': 4.104784,
 'd3d': 3.6692913}

With "Match" or "Match_Phrase"

In [40]:
match_test0002 = {"match" : {"text" : expanded["0002"]}}

In [41]:
ES_search("headers_standard", match_test0002)

{'a4c': 152.92398,
 'a6c': 149.1905,
 'a2': 80.02064,
 'a5b': 70.87577,
 'a5c': 63.691628,
 'a3': 62.509632,
 'a5d': 59.190556,
 'c3b': 57.04912,
 'd3d': 56.666798,
 'e4': 55.296684}

"Bool, should" and "Match"

In [45]:
match_test = {"bool":{"should":[{"match":{"text":{"query":expanded["0002"]}}}]}}

In [46]:
ES_search("hybrid_reindex", match_test)

{'a6c75': 95.89467,
 'a20': 90.73644,
 'e51': 89.76659,
 'a4c105': 86.93875,
 'a6c42': 81.72023,
 'a4c72': 80.15328,
 'a4c106': 77.97759,
 'a6c76': 75.50844,
 'a6d30': 73.30347,
 'a4c53': 72.953026}

Multiple Queries in One, Boosting

In [ ]:
"query":
{  
    "bool":{  
        "should":[  
            {  
                "match":{  
                    "title":{  
                        "query":"cat",
                        "boost":2
                    }
                }
            },
            {  
                "match":{  
                    "content":{  
                        "query":"cat",
                        "boost":2
                    }
                }
            }
        ]
    }
}

### Querying Many Comments

In [67]:
#es.indices.refresh(index=?)

In [126]:
def run_large_query(comment_dict, es_index):
    results = {}
    for key, value in comment_dict.items():
        query = {"match" : {"text" : value}}
        try:
            search1 = ES_search(es_index, query, 'dfs_query_then_fetch')
        except ElasticsearchException as es1:
            print(es1)
            es.indices.refresh(index=es_index)
            continue
        results[key] = {section:score for section, score in search1.items()}
        time.sleep(3)
        es.indices.refresh(index=es_index)
        print(key, end=' ')
    return results

In [16]:
fake_comments = {"matches_a2": "2. proposals for modified participation options under 5-year agreement periods\r\n     in developing the proposed policies described in this section, we considered a number of factors related to the program's current participation options in light of the program's financial results and stakeholders' feedback on program design, including the following.\r\n     first, we considered the program's existing policy allowing acos up to 6 years of participation in a one-sided model. we have found that the policy has shown limited success in encouraging acos to advance to performance-based risk. by the fifth year of implementing the program, only about 18 percent of the program's participating acos are under a two-sided model, over half of which are participating in the track 1+ model (see table 1).\r\n     as discussed in detail in the regulatory impact analysis (see section iv. of this proposed rule), our experience with the program indicates that acos in two-sided models generally perform better than acos that participate under a one-sided model. for example, for performance year 2016, about 68 percent of shared savings program acos in two-sided models (15 of 22 acos) shared savings compared to 29 percent of track 1 acos. for performance year 2015, prior to the first year of track 3, one of the three remaining track 2 acos shared savings, while about 30 percent of track 1 acos (118 of 389 acos) shared savings. for performance year 2014, two of the three remaining track 2 acos shared savings while about 25 percent of track 1 acos (84 of 330 acos) shared savings. in the program's first year, concluding december 31, 2013, 40 percent of track 2 acos (2 of 5 acos) compared to 23 percent of track 1 acos (50 of 215 acos) shared savings. see shared savings program accountable care organization public use files, available at https://www.cms.gov/research-statistics-data-and-systems/downloadable-public-use-files/sspaco/index.html. these observations, in combination with participation trends that show acos prefer to remain in track 1 for a second 3-year agreement period, suggests that a requirement for acos to more rapidly transition to performance-based risk could be effective in creating incentives for acos to more quickly meet the program's goals.     https://www.cms.gov/research-statistics-data-and-systems/downloadable-public-use-files/sspaco/index.html.\r\n     the program's current design lacks a sufficiently incremental progression to performance-based risk, the need for which is evidenced by robust participation in the new track 1+ model. we believe a significant issue that contributes to some acos' reluctance to participate in track 2 or track 3 is that the magnitude of potential losses is very high compared to the aco's degree of control over the total medicare parts a and b ffs expenditures for the aco's assigned beneficiaries, particularly when its aco participants have relatively low total medicare parts a and b ffs revenue. we are encouraged by the interest in the track 1+ model as indicated by the 55 shared savings program acos participating in the model for the performance year beginning on january 1, 2018; the largest group of shared savings program acos to enter into performance-based risk for a given performance year to date. based on the number of acos participating in the track 1+ model for performance year 2018, a lower risk option appears to be important for track 1 acos with experience in the program seeking to transition to performance-based risk, as well as acos seeking to enter an initial agreement period in the program under a lower risk model.\r\n     interest in the track 1+ model suggests that the opportunity to participate in an advanced apm while accepting more moderate levels of risk (compared to track 2 and track 3) is an important financial model design for acos. allowing more manageable levels of risk within the shared savings program is an important pathway for helping organizations to gain experience with managing risk as well as participating in advanced apms under the quality payment program. the high uptake we have observed with the track 1+ model also suggests that the current design of track 1 may be unnecessarily generous since the track 1+ model has the same level of upside as track 1 but under which acos must also assume performance-based risk.\r\n     second, under the program's current design, cms lacks adequate tools to properly address acos with patterns of negative financial performance. track 1 acos are not liable for repaying any portion of their losses to cms, and therefore may have potentially weaker incentives to improve quality and reduce growth in ffs expenditures within the accountable care model. these acos may take advantage of the potential benefits of continued program participation (including the receipt of program data and the opportunity to enter into certain contracting arrangements with aco participants and aco providers/suppliers in connection with their participation in the shared savings program), without providing a meaningful benefit to the medicare program. acos under two-sided models may similarly benefit from program participation and seek to continue their participation despite owing shared losses.\r\n     third, differences in performance of acos indicate a pattern where low revenue acos outperformed high revenue acos. as discussed in the regulatory impact analysis (see section iv. of this proposed rule), we have observed a pattern of performance, across tracks and performance years, where low revenue acos show better average results compared to high revenue acos. we believe high revenue acos, which typically include hospitals, have a greater opportunity to control assigned beneficiaries' total medicare parts a and b ffs expenditures, as they coordinate a larger portion of the assigned beneficiaries' care across care settings, and have the potential to perform better than what has been demonstrated in performance trends from 2012 through 2016. we conclude that the trends in performance by high revenue acos in relation to their expected capacity to control growth in expenditures are indications that these acos' performance would improve through greater incentives, principally a requirement to take on higher levels of performance-based risk, and thus drive change in ffs utilization for their medicare ffs populations. this conclusion is further supported by our initial experience with the track 1+ model, for which our preliminary findings support the conclusion that the degree of control an aco has over expenditures for its assigned beneficiaries is an indication of the level of performance-based risk an aco is prepared to accept and manage, where control is determined by the relationship between aco participants' total medicare parts a and b ffs revenue and the total medicare parts a and b ffs expenditures for the aco's assigned beneficiaries. our experience with the track 1+ model has also shown that aco participants' total medicare parts a and b ffs revenue as a percentage of the total medicare parts a and b ffs expenditures of the assigned beneficiaries can serve as a proxy for aco composition (that is, whether the aco includes one or more institutional providers as an aco participant, and therefore is likely to control a greater share of medicare parts a and b ffs expenditures and to have greater ability to coordinate care across settings for its assigned beneficiaries).\r\n     fourth, permitting choice of level of risk and assignment methodology within an aco's agreement period would create redundancy in some participation options, and eliminating this redundancy would allow cms to streamline the number of tracks offered while allowing acos greater flexibility to design their participation to meet the needs of their organizations. acos and stakeholders have indicated a strong preference for maintaining an option to select preliminary prospective assignment with retrospective reconciliation as an alternative to prospective assignment for acos under performance-based risk within the shared savings program. we considered what would occur if we retained track 2 in addition to the enhanced track and offered a choice of prospective assignment and preliminary prospective assignment (see section ii.a.4.c. of this proposed rule) for both tracks. we believe that acos prepared to accept higher levels of benchmark-based risk would be more likely to enter the enhanced track (which allows the greatest risk and potential reward). this is suggested by participation statistics, where 8 acos are participating in track 2 compared to the 38 acos participating in track 3 as of january 1, 2018. we note that for agreement periods beginning in 2018, only 2 acos entered track 2, both of which had deferred renewal in 2017, while 4 acos entered track 3 (for their first or second agreement period). acos may be continuing to pick track 2 because of the preliminary prospective assignment methodology, and we would expect participation in track 2 to decline further if we finalize the proposal to allow a choice of assignment methodology in the enhanced track, since we would expect acos ready for higher risk (that is, a level of risk that is higher than the highest level of risk and potential reward under the proposed basic track) to prefer the enhanced track over track 2.\r\n     fifth, longer agreement periods could improve program incentives and support acos' transition into performance-based risk when coupled with changes to improve the accuracy of the program's benchmarking methodology. extending agreement periods for more than 3 years could provide more certainty over benchmarks and in turn give acos a greater chance to succeed in the program by allowing them more time to understand their performance, gain experience and implement redesigned care processes before rebasing of the aco's historical benchmark. shared savings program results show that acos tend to perform better the longer they remain in the program. further, under longer agreement periods, historical benchmarks would become more predictable, since the benchmark would continue to be based on the expenditures for beneficiaries who would have been assigned to the aco in the 3 most recent years prior to the start of the aco's agreement period (see 425.602(a) and 425.603(c)) and the benchmark would be risk adjusted and updated each performance year relative to benchmark year 3. however, a number of factors can affect the amount of the benchmark, and therefore its predictability, during the agreement period regardless of whether the agreement period spans 3 or 5 years, including: adjustments to the benchmark during the aco's agreement period resulting from changes in the aco's certified aco participant list and regulatory changes to the assignment methodology; as well as variation in the benchmark value that occurs each performance year as a result of annual risk adjustment to the aco's benchmark (425.602(a)(9) and 425.603(c)(10)) and annual benchmark updates (425.602(b) and 425.603(d)). further, as discussed in section ii.d of this proposed rule, we believe the proposed approach to incorporating factors based on regional ffs expenditures in establishing, adjusting and updating the benchmark beginning with the aco's first agreement period will result in more accurate benchmarks. this improved accuracy of benchmarks would mitigate the impact of the more generous updated benchmarks that could result in the later years of longer agreement periods.\r\n     in summary, taking these factors into consideration, we propose to redesign the program's participation options by discontinuing track 1, track 2 and the deferred renewal option, and instead offering two tracks that eligible acos would enter into for an agreement period of at least 5 years: (1) basic track, which would include an option for eligible acos to begin participation under a one-sided model and incrementally phase-in risk (calculated based on aco participant revenue and capped at a percentage of the aco's updated benchmark) and potential reward over the course of a single agreement period, an approach referred to as a glide path;", \
                "matches_none": "the quick brown fox jumps over the lazy dog"}

In [27]:
mlt_test0002 = {"more_like_this": {"fields": ["text"], "like": fake_comments["matches_a2"], "min_term_freq": 1, "max_query_terms": 50}}

In [28]:
ES_search("headers_narrowsyn_custstop", mlt_test0002)

{'a2': 28.361528,
 'a3': 27.738836,
 'a5b': 26.990551,
 'a5d': 24.768711,
 'a7c': 23.503782,
 'd2': 21.002817,
 'a4c': 20.975582,
 'a6c': 20.871943,
 'e4': 20.871286,
 'a6d3': 20.378096}

In [20]:
run_large_query(fake_comments, "headers_narrowsyn_custstop")

matches_a2 matches_none 

{'matches_a2': {'a2': 962.5827,
  'a4c': 771.5503,
  'a6c': 722.64966,
  'a5b': 621.0664,
  'a3': 586.91016,
  'a5c': 560.27325,
  'c3b': 503.69507,
  'e4': 453.04593,
  'a7c': 425.9492,
  'a6b': 418.6685},
 'matches_none': {}}

In [160]:
# Test on dictionary of 3 key/value
dict_first3 = {}
for key in list(expanded.keys())[0:3]:
    dict_first3[key] = expanded[key]

run_large_query(dict_first3, "headers")

0002 0004 0005 

{'0002': {'a6c': 108.97458,
  'a4c': 107.97814,
  'd3d': 82.54391,
  'e5': 80.80219,
  'a5c': 76.095146,
  'd2': 74.08951,
  'c3b': 73.13041,
  'a5b': 70.770615,
  'a2': 59.04697,
  'c2': 58.028397},
 '0004': {'a4c': 57.26756,
  'a6c': 56.25602,
  'a5b': 48.102943,
  'e4': 44.462315,
  'a3': 40.867332,
  'e5': 39.244144,
  'c3b': 37.973938,
  'a2': 36.74947,
  'a5c': 36.44169,
  'e6': 34.330204},
 '0005': {'e5': 220.57083,
  'a4c': 210.88545,
  'a6c': 202.52736,
  'a5b': 197.70358,
  'c3b': 136.87021,
  'a5c': 125.184715,
  'd3d': 120.47933,
  'c2': 120.198006,
  'a2': 107.514725,
  'd2': 103.19107}}

### HEADERS CUSTOM Unique/clean comment Query

In [18]:
header_custom_results = run_large_query(expanded, "headers_custom")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [19]:
dump = json.dumps(header_custom_results)
f = open('match_header_custom_results.json','w')
f.write(dump)
f.close()

### HEADERS CUSTOM Unique/clean comment Query

In [32]:
headers_broadsyn_custstop_results = run_large_query(expanded, "headers_broadsyn_custstop")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [33]:
dump = json.dumps(headers_broadsyn_custstop_results)
f = open('headers_broadsyn_custstop_results.json','w')
f.write(dump)
f.close()

### HYBRID CUSTOM Unique/clean comment Query

In [20]:
hybrid_custom_results = run_large_query(expanded, "hybrid_custom")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [21]:
dump = json.dumps(hybrid_custom_results)
f = open('match_hybrid_custom_results.json','w')
f.write(dump)
f.close()

In [14]:
headers_custom_results = run_large_query(expanded, "headers_expsyn_basestop")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

In [15]:
dump = json.dumps(headers_custom_results)
f = open('headers_expsyn_basestop_results.json','w')
f.write(dump)
f.close()

In [35]:
headers_nosyn_basestop_results = run_large_query(expanded, "headers_nosyn_basestop")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

In [37]:
dump = json.dumps(headers_nosyn_basestop_results)
f = open('headers_nosyn_basestop_results.json','w')
f.write(dump)
f.close()

### comment syn included

In [22]:
headers_expsyn_comsyn_basestop_results = run_large_query(expanded, "headers_expsyn_comsyn_basestop")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

In [23]:
dump = json.dumps(headers_expsyn_comsyn_basestop_results)
f = open('headers_expsyn_comsyn_basestop_results.json','w')
f.write(dump)
f.close()

## Term Vectors 

#### Get top common words

In [53]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 30,
    "min_term_freq": 1,
    "min_doc_freq": 28 ## 90% of documents
  }
}

In [54]:
termvecs = dict()
result = es.termvectors(index="headers_1shard", id ='a2', body=termVectorBody)
{'a2':list(result['term_vectors']['text']['terms'].keys())}
for section in headers_id_list:
    result = es.termvectors(index = "headers_1shard", id = section, body= termVectorBody)
    termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [55]:
common_words = [value for key, value in termvecs.items()]
common_words = list(set([word for list1 in common_words for word in list1]))
common_words

['1',
 'particip',
 '2',
 'addit',
 '2019',
 'share',
 'provid',
 'cm',
 'perform',
 'includ',
 'beneficiari',
 'year',
 'rule',
 'program',
 "aco'",
 'propos',
 'final',
 'save',
 'aco',
 'subsequ',
 'requir',
 'establish',
 'section',
 '3',
 'appli']

#### Examine top words

In [20]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 10,
    "min_term_freq": 1,
    "min_doc_freq": 1,
    "max_doc_freq": 27
  }
}

In [21]:
termvecs = dict()
result = es.termvectors(index="headers_1shard", id ='a2', body=termVectorBody)
{'a2':list(result['term_vectors']['text']['terms'].keys())}
for section in headers_id_list:
    result = es.termvectors(index = "headers_1shard", id = section, body= termVectorBody)
    termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [22]:
%store -r boost_features

In [23]:
merged_dict = {key1:[value1, value2[1]] for (key1, value1), (key2, value2) in zip(termvecs.items(), boost_features.items())}
words = pd.DataFrame(data=merged_dict).T.rename({0:"rule_words", 1:"comment_words"}, axis=1).T
words = words.apply(lambda x: x.explode())
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
words

,a2,a3,a4b,a4c,a5b,a5c,a5d,a6b,a6c,a6d2,a6d3,a7b,a7c,b2a,b2b,c2,c3a,c3b,d2,d3b,d3c,d3d,d4,e2,e3,e4,e5,e6,e7
rule_words,agreement,base,agreement,adjust,assign,agreement,425.316,"5,000",adjust,applic,30,6,6,assign,1834,1899,425.312,approach,adjust,agreement,adjust,assign,425.601,425.402,99304,affect,d,advanc,arrang
rule_words,base,basic,basic,agreement,base,base,action,assign,agreement,dai,agreement,agreement,agreement,bed,act,425.304,care,assign,agreement,benchmark,aggreg,blend,425.603,act,99354,benchmark,fill,apm,care
rule_words,basic,benchmark,elect,assign,basic,basic,agreement,b,assign,decis,date,assign,assign,dai,furnish,act,data,base,assign,expenditur,cap,by3,425.800,assign,care,circumst,http,cehrt,coordin
rule_words,defer,level,glide,base,ff,enter,corridor,mlr,benchmark,end,effect,benchmark,juli,hospit,l,incent,declin,benchmark,benchmark,histor,enrol,compon,add,care,code,determin,measur,clinician,d
rule_words,level,limit,level,benchmark,high,model,financi,model,care,notic,liabl,calendar,list,preliminarili,origin,item,inform,care,cap,incorpor,percent,counti,adjust,clinician,cpt,disast,misus,criterion,data
rule_words,model,loss,path,period,low,period,monitor,msr,month,product,loss,determin,month,prospect,practition,m,medicar,claim,hcc,methodolog,region,factor,benchmark,design,evalu,extrem,opioid,elig,medic
rule_words,option,revenu,period,region,medicar,renew,neg,popul,period,quarter,month,expenditur,period,servic,prospect,payment,notic,methodolog,period,period,spend,nation,conform,physician,hcpc,ifc,part,meet,medicar
rule_words,period,reward,potenti,risk,revenu,risk,period,side,region,quarterli,portion,loss,qualiti,snf,servic,secretari,notif,opt,renorm,rebas,type,region,govern,primari,manag,qualiti,prescrib,payment,part
rule_words,risk,risk,reward,servic,total,termin,poor,track,servic,report,pro,month,report,swing,site,servic,primari,popul,risk,region,uncap,trend,refer,profession,primari,report,prescript,qualiti,pharmaci
rule_words,track,track,risk,track,track,track,track,variabl,track,typic,termin,period,sampl,waiver,telehealth,track,templat,primari,score,weight,weight,weight,subpart,specialti,servic,uncontrol,report,track,sponsor


## Sanity Check - synonyms

In [122]:
es.indices.delete(index='headers_expsyn_comsyn_basestop', ignore=[400, 404])

{'acknowledged': True}

In [111]:
fake_rule_chunks = [{"section": "a", "text": "AAA and other things and words"}, \
                    {"section": "b", "text": "hospitals and other things and words"}]
fake_split = rulesplit_toES(fake_rule_chunks, "sanity_check")

Last id uploaded: b


In [112]:
fake_comments = {"should_match_a": "abdominal aortic aneurysm", \
                 "should_match_neither": "This comment is not relevant to anything"}

In [113]:
run_large_query(fake_comments, "sanity_check")

should_match_a should_match_neither 

{'should_match_a': {'a': 0.6931471}, 'should_match_neither': {}}